J'essaie de scraper des infos sur la page de présentation Fragantica de Black Phantom : https://www.fragrantica.com/perfume/By-Kilian/Black-Phantom-43632.html

Je veux scraper ces infos :
Nom du Parfum
Nom de la marque
Launch year
Perfume rating
Nombre de votant sur le Perfume rating
Saisonnalité (Hiver, automne… peut etre gardé juste la + votée)
Parfumeur
Main accord
‘Perfume Pyramide’  = [‘Top note’ , ‘Middle note’, Base note’]
Genre (en prenant celui qui à le plus de vote)
Longevite, Sillage (en prenant les votes)


## Imports

In [6]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import cloudscraper

import numpy as np
import pandas as pd


## Les fonctions

### DESROMAIS INUTILE fonction pour récuperer le code html d'une page parfum

In [2]:
#Récupère le HTML de la page correspondant à l'URL donnée.
def get_html_headless(url):
    """
    Ouvre Chrome en mode headless, va sur l'URL, 
    attend x seconde, récupère le HTML.
    """

    #1) ✅ Configuration Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Désactiver l'ouverture de Chrome
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--blink-settings=imagesEnabled=false") # Pas besoin d'images = gain de temps
    options.add_argument("--headless=new")  # Nouvelle version plus rapide du headless
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

    # 2) Création du driver
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options)
        
    # 1) Accreder à l'URL
    driver.get(url)
    
    # 3) Récupère le code HTML
    html_content = driver.page_source

    driver.quit()

    return html_content

### Fonctions intermédiaire pour récuperer les infos sur un parfum

In [2]:
# 1) Nom du parfum
def extract_perfume_name(html_content):
    """Extrait le nom du parfum depuis le contenu HTML"""
    match = re.search(r'/perfume/[^/]+/([^/]+)-\d+\.html', html_content)
    return match.group(1).replace('-', ' ') if match else None

# 2) Marque
def extract_brand_name(html_content):
    """Extrait la marque du parfum depuis le contenu HTML"""
    match = re.search(r'/perfume/([^/]+)/', html_content)
    return match.group(1).replace('-', ' ') if match else None

#3) Parfumeur
def extract_nose(soup):
    """Extrait le nom du parfumeur"""
    nose_el = soup.select_one('div.cell a[href^="/noses/"]')
    return nose_el.get_text(strip=True) if nose_el else None

#4) Année de sortie
def extract_launch_year(soup):
    """Extrait l'année de sortie du parfum"""
    title_el = soup.find('title')
    if title_el:
        possible_year = title_el.get_text(strip=True)[-4:]
        if possible_year.isdigit():
            return possible_year
    return None

#5) Perfume rating
def extract_rating(soup):
    """Extrait la note du parfum"""
    rating_el = soup.select_one('span[itemprop="ratingValue"]')
    return rating_el.get_text(strip=True) if rating_el else None

#6) Nombre de votes
def extract_rating_count(soup):
    """Extrait le nombre de votes"""
    rating_count_el = soup.select_one('span[itemprop="ratingCount"]')
    return rating_count_el.get_text(strip=True) if rating_count_el else None

#7) Accords principaux
def extract_main_accords(soup):
    """Extrait les accords principaux du parfum"""
    main_accords_el = soup.find_all('div', class_='cell accord-box')
    return [element.get_text(strip=True) for element in main_accords_el if element.get_text(strip=True)]

#8) Genre
def extract_gender(gender_list):
    """Détermine le genre du parfum en fonction des votes"""
    if not gender_list:
        return None
    
    sum_votes = np.sum([int(element[1]) for element in gender_list])
    
    if sum_votes > 8:
        female_count = int(gender_list[0][1]) + int(gender_list[1][1])
        male_count = int(gender_list[3][1]) + int(gender_list[4][1])
        unisex_count = 1.2 * int(gender_list[2][1])

        if female_count > male_count and female_count > unisex_count:
            return "female"
        elif male_count > unisex_count:
            return "male"
        else:
            return "unisex"
    return None

#9) Longévité
def extract_longevity(longevity_list):
    """Détermine la longévité du parfum"""
    if not longevity_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in longevity_list])
    if sum_votes > 8:
        longevity = np.argmax([int(element[1]) for element in longevity_list])
        return longevity_list[longevity][0]
    return None

#10) Sillage
def extract_sillage(sillage_list):
    """Détermine le sillage du parfum"""
    if not sillage_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in sillage_list])
    if sum_votes > 8:
        sillage = np.argmax([int(element[1]) for element in sillage_list])
        return sillage_list[sillage][0]
    return None

#11) Ressenti du prix
def extract_price_feeling(price_feeling_list):
    """Détermine le ressenti du prix du parfum"""
    if not price_feeling_list:
        return None

    sum_votes = np.sum([int(element[1]) for element in price_feeling_list])
    if sum_votes > 8:
        price_feeling = np.argmax([int(element[1]) for element in price_feeling_list])
        return price_feeling_list[price_feeling][0]
    return None

#12) Pyramide olfactive
def extract_pyramid_ingredients(soup, pyramid_section):
    """Extrait les ingrédients d'une section de la pyramide olfactive"""
    pyramid_ingredients = []
    #On repère la bonne zone grâce à la section de la pyramide
    header = soup.find('h4', string=lambda text: text and pyramid_section in text)
    if header:
        #Les noms des ingrédients sont stockés dans la div suivante
        div = header.find_next('div')
        if div:
            ingredients_divs = div.find_all('div')
            for ingredient_div in ingredients_divs:
                a_tag = ingredient_div.find('a')
                if a_tag and a_tag.next_sibling:
                    ingredient_name = a_tag.next_sibling.strip()
                    if ingredient_name not in pyramid_ingredients:
                        pyramid_ingredients.append(ingredient_name)
    return pyramid_ingredients

### Fonctions intermediaire pour le tableau avec Gender, Sillage, loongevity et Price feeling

In [4]:
def extract_votes(soup):
    """Extrait les catégories et leurs votes à partir du HTML."""
    tableau_votes = []
    tab_rows = soup.select('div.grid-x.grid-margin-x')  # Sélection des blocs contenant les votes
    
    for row in tab_rows:
        category_el = row.select_one('span.vote-button-name')  # Nom de la catégorie
        votes_el = row.select_one('span.vote-button-legend')  # Nombre de votes

        if category_el and votes_el:
            category = category_el.get_text(strip=True)
            votes = votes_el.get_text(strip=True)

            # Vérifier si le nombre de votes est bien un chiffre
            if votes.isdigit():
                tableau_votes.append((category, votes))

    return tableau_votes

def filter_votes(tableau_votes, labels, remove_first=False, remove_last=False):
    """
    Filtre les votes selon un ensemble de labels et enlève potentiellement le premier ou dernier élément.
    
    :param tableau_votes: Liste des votes [(catégorie, nombre)]
    :param labels: Ensemble des labels à filtrer
    :param remove_first: Supprime le premier élément si True
    :param remove_last: Supprime le dernier élément si True
    :return: Liste filtrée et éventuellement modifiée
    """
    filtered_list = [element for element in tableau_votes if element[0] in labels]

    if remove_first and filtered_list:
        filtered_list.pop(0)
    if remove_last and filtered_list:
        filtered_list.pop()
    
    return filtered_list

### Scrape perfume info

In [3]:
def scrape_perfume_info(html_content, soup):
    """
    Scrap les infos principales depuis la page Fragrantica d'un parfum.
    Retourne un dictionnaire avec ces informations.
    """
    
    
    perfume_data = {
        "nom_parfum": extract_perfume_name(html_content),
        "marque": extract_brand_name(html_content),
        "nose": extract_nose(soup),
        "launch_year": extract_launch_year(soup),
        "rating_value": extract_rating(soup),
        "rating_count": extract_rating_count(soup),
    }

    # 3) Retourner le dictionnaire
    return perfume_data

## Brouillon

In [21]:
from bs4 import BeautifulSoup
import time
import timeit
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
from selenium.webdriver.common.proxy import Proxy, ProxyType


import pandas as pd
import numpy as np


# Fonction pour lire les proxies depuis un fichier
def read_proxies(file_path):
    with open(file_path, 'r') as file:
        proxies = file.readlines()
    return [proxy.strip() for proxy in proxies]


#~~~~~~~~~~~~~~~~~~~~~~~~~~~FONCTION PRINCIPALE~~~~~~~~~~~~~~~~~~~~~~~~~~~#
def scraping_multi_perfume_info_1 (list_url):
    """
    Scrap les informations de plusieurs parfums depuis une liste d'URLs.
    Retourne un DataFrame avec ces informations.
    """
    all_data = [] 
    ua = UserAgent()

    #✅ Configuration Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Désactiver l'ouverture de Chrome
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--blink-settings=imagesEnabled=false") # Pas besoin d'images = gain de temps
    options.add_argument("--headless=new")  # Nouvelle version plus rapide du headless
    options.add_argument(f"user-agent={ua.random}")  # Utiliser un user-agent 

    # Utiliser un proxy

    # Choisir un proxy aléatoire
    proxies = read_proxies('proxies.txt')
    proxy_ip_port = random.choice(proxies)
    print(f"Utilisation du proxy : {proxy_ip_port}")
    
    proxy = Proxy()
    proxy.proxy_type = ProxyType.MANUAL
    proxy.http_proxy = proxy_ip_port
    proxy.ssl_proxy = proxy_ip_port
    options.proxy = proxy

    for url in list_url :
        # 2) Création du driver
        driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options)

        # 3) Récupération du contenu HTML
        driver.get(url)
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        driver.quit()

        # Vérifier si la page a été chargée correctement
        if "429 Too Many Requests" in html_content:
            print(f"⚠️ Trop de requêtes pour l'URL : {url}")
            continue


        # On appelle la fonction qui fait le scraping pour UNE URL
        perfume_info = scrape_perfume_info_1(html_content, soup)
        # On ajoute le résultat (dict) à la liste all_data
        all_data.append(perfume_info)

    # Convertir la liste de dicts en DataFrame
    all_data_df = pd.DataFrame(all_data)
    return all_data_df
 

def scraping_multi_perfume_info_2 (list_url):
    """
    Scrap les informations de plusieurs parfums depuis une liste d'URLs.
    Retourne un DataFrame avec ces informations.
    """
    all_data = [] 

    #✅ Configuration Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Désactiver l'ouverture de Chrome
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--blink-settings=imagesEnabled=false") # Pas besoin d'images = gain de temps
    options.add_argument("--headless=new")  # Nouvelle version plus rapide du headless
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

    for url in list_url :
        # 2) Création du driver
        driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options)

        # 3) Récupération du contenu HTML
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()

        # On appelle la fonction qui fait le scraping pour UNE URL
        perfume_info = scrape_perfume_info_2(soup)
        # On ajoute le résultat (dict) à la liste all_data
        all_data.append(perfume_info)

    # Convertir la liste de dicts en DataFrame
    all_data_df = pd.DataFrame(all_data)
    return all_data_df
 


#~~~~~~~~~~~~~~~~~~~~~~~~~~~FONCTION SECONDAIRE~~~~~~~~~~~~~~~~~~~~~~~~~~~# 
def scrape_perfume_info_1(html_content, soup):
    """
    Scrap les infos principales depuis la page Fragrantica d'un parfum.
    Retourne un dictionnaire avec ces informations.
    """
   
    # 2) Extraire les données (nom, marque, etc.)
    votes_dict = extract_all_votes(soup)
    nom, marque = extract_name_and_brand(soup)

    perfume_data = {
        "nom_parfum": nom,
        "marque": marque,
        "nose": extract_nose(soup),
        "launch_year": extract_launch_year(soup),
        "rating_value": extract_rating(soup),
        "rating_count": extract_rating_count(soup),
        "main_accords": extract_main_accords(soup),
        "gender": extract_gender(votes_dict),
        "longevity": extract_longevity(votes_dict),
        "sillage": extract_sillage(votes_dict),
        "price_feeling": extract_price_feeling(votes_dict),    
        "top_notes": extract_pyramid_ingredients(soup, "Top Notes"),
        "middle_notes": extract_pyramid_ingredients(soup, "Middle Notes"),
        "base_notes": extract_pyramid_ingredients(soup, "Base Notes")
    }

    # 3) Retourner le dictionnaire
    return perfume_data

def scrape_perfume_info_2(soup):
    """
    Scrap les infos principales depuis la page Fragrantica d'un parfum.
    Retourne un dictionnaire avec ces informations.
    """
   
    # 2) Extraire les données (nom, marque, etc.)
    votes_dict = extract_all_votes(soup)
    nom, marque = extract_name_and_brand(soup)

    perfume_data = {
        "nom_parfum": nom,
        "marque": marque,
        "nose": extract_nose(soup),
        "launch_year": extract_launch_year(soup),
        "rating_value": extract_rating(soup),
        "rating_count": extract_rating_count(soup),
        "main_accords": extract_main_accords(soup),
        "gender": extract_gender(votes_dict),
        "longevity": extract_longevity(votes_dict),
        "sillage": extract_sillage(votes_dict),
        "price_feeling": extract_price_feeling(votes_dict),    
        "top_notes": extract_pyramid_ingredients(soup, "Top Notes"),
        "middle_notes": extract_pyramid_ingredients(soup, "Middle Notes"),
        "base_notes": extract_pyramid_ingredients(soup, "Base Notes")
    }

    # 3) Retourner le dictionnaire
    return perfume_data



#~~~~~~~~~~~~~~~~~~~~~~~~~~~FONCTIONS INTERMEDIAIRES~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# &) Intermédiare d'intermédiaire
def extract_all_votes(soup):
    """
    Extrait tous les votes de la page sous forme d'un dictionnaire.
    
    :param soup: Objet BeautifulSoup contenant le HTML.
    :return: Dictionnaire {'category': votes}
    """
    votes_dict = {}
    seen_moderate = 0

    # Sélectionner tous les blocs contenant les votes
    vote_rows = soup.select('div.grid-x.grid-margin-x')

    for row in vote_rows:
        category_el = row.select_one('span.vote-button-name')
        votes_el = row.select_one('span.vote-button-legend')

        if category_el and votes_el:
            category = category_el.get_text(strip=True)
            votes = votes_el.get_text(strip=True)

            if category == "moderate":
                    seen_moderate += 1
                    if seen_moderate == 2:
                        category = "average"

            # Vérifier si le vote est bien un nombre    
            if votes.isdigit():
                votes_dict[category] = int(votes)
            

    return votes_dict

# 1-2) Nom du parfum & Marque
def extract_name_and_brand(soup):
    """
    Extrait le nom du parfum et la marque depuis la balise meta 'keywords'.
    """
    meta_keywords = soup.find("meta", {"name": "keywords"})
    if meta_keywords:
        content = meta_keywords.get("content", "")
        parts = [p.strip() for p in content.split(",")]  # Nettoyer les espaces
        name = parts[0] if parts else None
        brand = parts[1] if len(parts) > 1 else None
        return name, brand
    return None, None

#3) Parfumeur
def extract_nose(soup):
    """Extrait le nom du parfumeur"""
    nose_el = soup.select_one('div.cell a[href^="/noses/"]')
    return nose_el.get_text(strip=True) if nose_el else None

#4) Année de sortie
def extract_launch_year(soup):
    """Extrait l'année de sortie du parfum"""
    launch_year = soup.title.text.strip()[-4:] if soup.title else ""
    return launch_year if launch_year.isdigit() else None

#5) Perfume rating
def extract_rating(soup):
    """Extrait la note du parfum"""
    rating_el = soup.select_one('span[itemprop="ratingValue"]')
    return rating_el.get_text(strip=True) if rating_el else None

#6) Nombre de votes
def extract_rating_count(soup):
    """Extrait le nombre de votes"""
    rating_count_el = soup.select_one('span[itemprop="ratingCount"]')
    return rating_count_el.get_text(strip=True) if rating_count_el else None

#7) Accords principaux
def extract_main_accords(soup):
    """Extrait les accords principaux du parfum"""
    main_accords_el = soup.find_all('div', class_='cell accord-box')
    return [element.get_text(strip=True) for element in main_accords_el if element.get_text(strip=True)]

#8) Genre
def extract_gender(votes_dict):
    """Détermine le genre du parfum en fonction des votes extraits."""
    gender_labels = ["female", "more female", "unisex", "more male", "male"]
    gender_votes = [votes_dict.get(label, 0) for label in gender_labels]
    
    if sum(gender_votes) <= 8:
        return None

    female_count = gender_votes[0] + gender_votes[1]
    male_count = gender_votes[3] + gender_votes[4]
    unisex_count = 1.2 * gender_votes[2]

    return max(("female", female_count),("male", male_count),("unisex", unisex_count),key=lambda x: x[1])[0]

#9) Longévité
def extract_longevity(votes_dict):
    """Détermine la longévité dominante en fonction des votes."""
    longevity_labels = ["very weak", "weak", "moderate", "long lasting", "eternal"]
    longevity_votes = [votes_dict.get(label, 0) for label in longevity_labels]
    
    return max(zip(longevity_labels, longevity_votes), key=lambda x: x[1])[0] if sum(longevity_votes) > 8 else None

#10) Sillage
def extract_sillage(votes_dict):
    """Détermine le sillage dominant en fonction des votes."""
    sillage_labels = ["intimate", "average", "strong", "enormous"]
    sillage_votes = [votes_dict.get(label, 0) for label in sillage_labels]

    return max(zip(sillage_labels, sillage_votes), key=lambda x: x[1])[0] if sum(sillage_votes) > 8 else None

#11) Perception du prix
def extract_price_feeling(votes_dict):
    """Détermine la perception du prix en fonction des votes."""
    price_labels = ["way overpriced", "overpriced", "ok", "good value", "great value"]
    price_votes = [votes_dict.get(label, 0) for label in price_labels]

    return max(zip(price_labels, price_votes), key=lambda x: x[1])[0] if sum(price_votes) > 8 else None

#12) Pyramide olfactive
## Par sous partie de la pyramide (notes de tête, de coeur, de fond)
def extract_pyramid_ingredients(soup, pyramid_section):
    """Extrait les ingrédients d'une section de la pyramide olfactive"""
    header = soup.find('h4', string=lambda text: text and pyramid_section in text)
    if not header:
        return []

    # Chercher directement tous les <a> après le header
    div = header.find_next('div')
    return list({a_tag.next_sibling.strip() for a_tag in div.find_all('a') if a_tag.next_sibling}) if div else []


In [1]:
urls = [
        "https://www.fragrantica.com/perfume/By-Kilian/Black-Phantom-43632.html",
        "https://www.fragrantica.com/perfume/Mancera/Wild-Leather-28084.html",
        "https://www.fragrantica.com/perfume/Ministry-of-Oud/Oud-Satin-74588.html"]


In [22]:
df2 = scraping_multi_perfume_info_1(urls)
df2.head()

Utilisation du proxy : 27.79.239.226:16000


,nom_parfum,marque,nose,launch_year,rating_value,rating_count,main_accords,gender,longevity,sillage,price_feeling,top_notes,middle_notes,base_notes
0,None,None,None,None,None,None,[],None,None,None,None,[],[],[]
1,None,None,None,None,None,None,[],None,None,None,None,[],[],[]
2,None,None,None,None,None,None,[],None,None,None,None,[],[],[]


## Idée d'améliorations

In [ ]:

# Utilisation de lambda pour éviter l'exécution immédiate
old_time = timeit.timeit(lambda: scrape_perfume_info_1(html_content,soup), number=100)
new_time = timeit.timeit(lambda: scrape_perfume_info_2(soup), number=100)

print(f"⏳ Ancienne fonction : {old_time:.5f} sec")
print(f"🚀 Nouvelle fonction : {new_time:.5f} sec")
print(f"💨 Gain de temps : {((old_time - new_time) / old_time) * 100:.2f}%")



'''

--> CREER une seul fonction qui n'appele pas d'autre fonction
'''
print("FIN")

⏳ Ancienne fonction : 9.30231 sec
🚀 Nouvelle fonction : 9.21441 sec
💨 Gain de temps : 0.95%
FIN
